In [23]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation, Conv1D, MaxPooling1D, Masking, Bidirectional
%matplotlib inline

In [24]:
train_df = pd.read_csv('../LSTM/data/processed_train.csv')
td = pd.read_csv('../LSTM/data/processed_test.csv')

In [25]:
td

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s19,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6,cycle_norm
0,1,1,0.238019,0.297150,1.0,0.597937,0.639243,0.691329,0.680395,0.617180,...,1.0,0.625944,0.630724,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,2,0.476162,0.831354,1.0,0.626985,0.664861,0.647519,0.534663,0.507937,...,1.0,0.484214,0.496301,0.0,0.0,0.0,0.0,1.0,0.0,0.002732
2,1,3,0.833282,0.997625,1.0,0.060269,0.189055,0.312512,0.243225,0.146592,...,1.0,0.164722,0.159476,0.0,0.0,0.0,1.0,0.0,0.0,0.005464
3,1,4,0.999967,0.998812,1.0,0.000000,0.135525,0.286753,0.242061,0.000000,...,1.0,0.005834,0.016866,0.0,1.0,0.0,0.0,0.0,0.0,0.008197
4,1,5,0.595089,0.737886,0.0,0.238089,0.012028,0.022001,0.050986,0.293184,...,0.0,0.141730,0.140123,0.0,0.0,1.0,0.0,0.0,0.0,0.010929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33986,259,119,0.833210,0.997981,1.0,0.060269,0.185015,0.331523,0.245972,0.146592,...,1.0,0.163349,0.159133,0.0,0.0,0.0,1.0,0.0,0.0,0.322404
33987,259,120,0.999967,0.998219,1.0,0.000000,0.128638,0.290703,0.232087,0.000000,...,1.0,0.013384,0.016260,0.0,1.0,0.0,0.0,0.0,0.0,0.325137
33988,259,121,0.999955,0.997625,1.0,0.000000,0.130750,0.285244,0.219835,0.000000,...,1.0,0.013384,0.020433,0.0,1.0,0.0,0.0,0.0,0.0,0.327869
33989,259,122,0.000057,0.000356,1.0,1.000000,0.984024,0.944915,0.935747,1.000000,...,1.0,0.991764,0.985410,0.0,0.0,0.0,0.0,0.0,1.0,0.330601


In [26]:
train_df.loc[train_df['RUL']>130,['RUL']]=130

In [27]:
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6,RUL,cycle_norm
0,1,1,0.833134,0.997625,1.0,0.060269,0.181576,0.311201,0.273095,0.146592,...,0.156036,0.159082,0.0,0.0,0.0,1.0,0.0,0.0,130,0.000000
1,1,2,0.999767,0.998575,1.0,0.000000,0.131847,0.296600,0.245535,0.000000,...,0.007888,0.014562,0.0,1.0,0.0,0.0,0.0,0.0,130,0.002653
2,1,3,0.595096,0.738480,0.0,0.238089,0.016332,0.035297,0.056997,0.293184,...,0.133745,0.151414,0.0,0.0,1.0,0.0,0.0,0.0,130,0.005305
3,1,4,0.999993,0.999525,1.0,0.000000,0.128269,0.298795,0.246979,0.000000,...,0.014060,0.026144,0.0,1.0,0.0,0.0,0.0,0.0,130,0.007958
4,1,5,0.595137,0.736698,0.0,0.238089,0.014130,0.037871,0.058152,0.293184,...,0.135460,0.143240,0.0,0.0,1.0,0.0,0.0,0.0,130,0.010610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,0.476188,0.831354,1.0,0.626985,0.672172,0.682297,0.591489,0.507937,...,0.486283,0.483993,0.0,0.0,0.0,0.0,1.0,0.0,4,0.824934
53755,260,313,0.238102,0.298100,1.0,0.597937,0.644830,0.733008,0.722934,0.617180,...,0.614540,0.622022,1.0,0.0,0.0,0.0,0.0,0.0,3,0.827586
53756,260,314,0.595222,0.736342,0.0,0.238089,0.017892,0.088067,0.082198,0.293184,...,0.137517,0.144474,0.0,0.0,1.0,0.0,0.0,0.0,2,0.830239
53757,260,315,0.595203,0.738717,0.0,0.238089,0.021195,0.079155,0.102368,0.293184,...,0.132716,0.134383,0.0,0.0,1.0,0.0,0.0,0.0,1,0.832891


In [28]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191

In [29]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3','cycle_norm','mode1','mode2','mode3','mode4','mode5','mode6']
# sequence_cols = ['setting1', 'setting2', 'setting3','cycle_norm']
sequence_cols.extend(sensor_cols)

In [30]:
sequence_length = 20

In [31]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(48559, 20, 31)

In [32]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [33]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)


In [34]:
import keras.backend as K

In [35]:
def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [40]:
nb_features

31

In [42]:
# Build model
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(Conv1D(filters=1, kernel_size=3,input_shape=(sequence_length, nb_features)))
model.add(Conv1D(filters=1, kernel_size=3))
model.add(LSTM(
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])
# model.compile(loss=scoring_error, optimizer='rmsprop',metrics=['mae',r2_keras])

print(model.summary())


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 18, 1)             94        
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 16, 1)             4         
_________________________________________________________________
lstm_9 (LSTM)                (None, 16, 100)           40800     
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 50)                0         
__________________________

In [43]:
# fit the network
model_path='convlstm.h5'
history = model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )

# list all data in history
print(history.history.keys())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 46131 samples, validate on 2428 samples
Epoch 1/100
 - 34s - loss: 6727.9959 - mean_absolute_error: 70.7977 - r2_keras: -2.5944e+00 - val_loss: 6406.5465 - val_mean_absolute_error: 69.4128 - val_r2_keras: -2.9207e+00
Epoch 2/100
 - 30s - loss: 5237.1171 - mean_absolute_error: 61.7193 - r2_keras: -1.7974e+00 - val_loss: 5000.6577 - val_mean_absolute_error: 61.0297 - val_r2_keras: -2.1064e+00
Epoch 3/100
 - 32s - loss: 4050.9947 - mean_absolute_error: 54.3481 - r2_keras: -1.1650e+00 - val_loss: 3860.7335 - val_mean_absolute_error: 53.9672 - val_r2_keras: -1.4994e+00
Epoch 4/100
 - 30s - loss: 3135.4017 - mean_absolute_error: 48.4172 - r2_keras: -6.7189e-01 - val_loss: 2984.3538 - val_mean_absolute_error: 48.2137 - val_r2_keras: -1.0975e+00
Epoch 5/100
 - 31s - loss: 2476.8248 - mean_absolute_error: 43.8643 - r2_keras: -3.2077e-01 - val_loss: 2368.6951 - val_mean_absolute_error: 43.7618

KeyboardInterrupt: 

In [ ]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Loss: {}'.format(scores[0]))

In [ ]:
test = []
for i in range(len(td.groupby('id'))):
    if len(td[td['id']==(i+1)]) >= sequence_length:
        test.append(np.asarray(td[td['id']==(i+1)][sequence_cols].values[-sequence_length:]).astype(np.float32).tolist())
    else:
        test.append(np.asarray(td[td['id']==(i+1)][sequence_cols].values[:]).astype(np.float32).tolist())

In [ ]:
padded_inputs = keras.preprocessing.sequence.pad_sequences(test,dtype='float32',                                padding='post')

In [ ]:
result = model.predict(padded_inputs,verbose=1)

In [ ]:
submit = pd.read_excel('../results_submission.xls')
submit['RUL prediction']=result
submit.to_csv('deeplstm.csv')

In [ ]:
ans = np.loadtxt('../RUL_FD002.txt')
ans.shape

In [ ]:
def score(user_ans):
    user_ans_arr = np.array(user_ans)
    score = 0
    diff = user_ans_arr - ans
    diff_pos = diff[diff>=0]
    diff_neg = diff[diff<0]
    score = np.sum((np.exp(-1./13.*diff_neg)-1))+np.sum((np.exp(1./10.*diff_pos)-1))
    return score

In [ ]:
re = pd.read_csv('deeplstm.csv')
pred_rul=re['RUL prediction']
score(pred_rul)